## <center> Homework 7 </center>
<center> Tara Wilson </center>
<center> DATA 558 </center>
<center> May 25, 2019 </center>

In [1]:
# imports
import numpy as np
import pandas as pd
import os
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from scipy.linalg import eigh
from numpy.linalg import inv
import random
import copy 
from sklearn.datasets import load_digits
import warnings
warnings.filterwarnings('ignore')

### Exercise 1

**Study Lab #7 and do exercises therin**  

See seperate submission attachment.

### Exercise 2

**In this problem you will implement in Python a first version of your own kernerl support vector machine with the smoothed hinge loss.**  

The kernel support vector maching with the smoothed hinge loss writes as
$min F(\alpha) := \frac {1}{n} \sum_{i=1}^{n} l_{hh}(y_i,(k\alpha)_i) + \lambda \alpha^T K \alpha$    

where $K(\alpha)_i$ is the *i*th entry in the vector $K\alpha$,  

$\begin{equation}
l_{hh}(y, t) := \begin{cases}
    0 \text{    if    } yt > 1 + h \\  
    \frac{(1+h-yt)^2}{4h} \text{    if    } |1 - yt| \leq h \\  
    1 - yt \text{    if    } yt < 1 - h  
    \end{cases}
\end{equation}$  

and h = 0.5

**Compute the gradient $\nabla(\alpha)$ of F**

**Write a function _computegram_ that computes, for any set of datapoints $x_1, . . . , x_n$, the kernel matrix K**

In [ ]:
def computegram(x1, x2):
    

**Write a function kerneleval that computes, for any set of datapoints $x_1, . . . , x_n$ and a new datapoint $x^*$, the vector of kernel evaluations $[k(x_1,x^*), ..., k(x_n, x^*)]^T$**

**Consider the `Digits` dataset (http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html). Download and standardize the data, if you have not done so already**

In [ ]:
# download the data
XDigits, yDigits = load_digits(return_X_y = True)
yDigits = yDigits.reshape((len(yDigits), 1))
print("Shape of the digits X data:", XDigits.shape)
print("Shape of the digits y data:", yDigits.shape)

In [ ]:
# standardize X data
scaler = preprocessing.StandardScaler().fit(XDigits)
XDigits = scaler.transform(XDigits)

# standardizing y labels
scaler = preprocessing.StandardScaler().fit(yDigits)
yDigits = scaler.transform(yDigits)

**Write a function _mysvm_ that implements the fast gradient algorithm to train the kernel support vector machine with the smoothed hinge loss. The function takes as input the initial step-size value for the backtracking rule and a stopping criterion based on the norm of the gradient**

**Train your kernel support vector machine with the smoothed hinge loss and the polynomial kernel of order 7 on the the `Digits` dataset, tuning the regularization parameter λ using cross-validation. The *p*th order polynomial kernel is given by $k(x, y) = (x^Ty+b)^P$. You may take b = 1.**

**Compare the performance of kernel SVMs with different kernels (polynomial kernels with different orders, Gaussian RBF with different bandwidths, etc.).**

### Exercise 3: Data Competition Project

**In this exercise, you are going to train support vector machines (SVMs) using the data competition 2 project dataset (with 100 classes). You will consider here all classes in the dataset. You may work on this exercise on your own computer first. Note, however, that you need AWS to run the experiments for this entire exercise.**

In [10]:
data_dir = "../Kaggle/data558spring2019-competition2"

X_train = np.load(os.path.join(data_dir, 'train_features.npy'))
y_train = np.load(os.path.join(data_dir, 'train_labels.npy')).reshape(-1, 1)
X_test = np.load(os.path.join(data_dir, 'val_features.npy'))
y_test = np.load(os.path.join(data_dir, 'val_labels.npy')).reshape(-1, 1)

# standardize X data
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# standardize y labels
# scaler = preprocessing.StandardScaler().fit(y_train)
# y_train = scaler.transform(y_train)
# y_test = scaler.transform(y_test)

print("Dimension of X_train:", X_train.shape)
print("Dimension of y_train:", y_train.shape)
print("Dimension of X_test:", X_test.shape)
print("Dimension of y_test:", y_test.shape)

Dimension of X_train: (50000, 4096)
Dimension of y_train: (50000, 1)
Dimension of X_test: (10000, 4096)
Dimension of y_test: (10000, 1)


**In a one-vs-one fashion, for each pair of classes, train a linear SVM classifier using scikitlearn’s function `LinearSVC`, with the default value for the regularization parameter. Compute the *multi-class misclassification error* obtained using these classifiers trained
in a one-vs-one fashion**

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


KeyboardInterrupt: 

**In a one-vs-rest fashion, for each class, train a linear SVM classifier using scikit-learn’s function `LinearSVC`, with the default value for $\lambda_c$. Compute the multi-class misclassification error obtained using these classifiers trained in a one-vs-rest fashion.**

In [ ]:
from sklearn.svm import LinearSVC

for label in np.unique(y_train):
    print(label)
    y_train_temp = copy.deepcopy(y_train)
    y_train_temp[y_train_temp == label] = -1
    y_train_temp[y_train_temp != -1] = 1
    clf = LinearSVC(random_state=0, tol=1)
    clf.fit(X_train, y_train_temp).get_params
    

**Redo all questions above now using your own code for the linear SVMs from Exercise 1. Make to sure to run preliminary experiments to decide how to set the stopping criterion to a value that allows the experiments to complete in a reasonable amount of time.**